In [1]:
import pandas as pd
import re

In [2]:
# Leitura do arquivo de alunos; arquivo extraído do BigQuery
alunos = pd.read_csv('alunos_usp_000000000000.csv')

In [3]:
# Checagem
alunos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2783 entries, 0 to 2782
Data columns (total 12 columns):
estado              2783 non-null object
cidade              2783 non-null object
ies                 2783 non-null object
campus              2783 non-null object
curso               2783 non-null object
grau_turno          2783 non-null object
modalidade          2783 non-null object
colocacao           2783 non-null object
nome                2783 non-null object
codigo_inscricao    2783 non-null object
nota                2783 non-null float64
bonus               0 non-null float64
dtypes: float64(2), object(10)
memory usage: 152.2+ KB


In [4]:
# Remoção de possíveis linhas repetidas
alunos.drop_duplicates(inplace=True)

In [5]:
# Checagem
alunos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2765 entries, 0 to 2782
Data columns (total 12 columns):
estado              2765 non-null object
cidade              2765 non-null object
ies                 2765 non-null object
campus              2765 non-null object
curso               2765 non-null object
grau_turno          2765 non-null object
modalidade          2765 non-null object
colocacao           2765 non-null object
nome                2765 non-null object
codigo_inscricao    2765 non-null object
nota                2765 non-null float64
bonus               0 non-null float64
dtypes: float64(2), object(10)
memory usage: 172.8+ KB


Temos __2.765 alunos__ que entraram na USP com a nota do Sisu.

In [6]:
# Leitura do arquivo de vagas; dados extraídos do BigQuery
vagas = pd.read_csv('vagas_usp_000000000000.csv')

In [7]:
# Checagem
vagas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 9 columns):
uf                 512 non-null object
ies                512 non-null object
campus             512 non-null object
curso              512 non-null object
grau               512 non-null object
turno              512 non-null object
numero_vagas       512 non-null int64
tipo_modalidade    512 non-null object
modalidade         512 non-null object
dtypes: int64(1), object(8)
memory usage: 20.0+ KB


In [8]:
# Normalização do curso para poder fundir com o df 'alunos' posteriormente
vagas['cod_temp'] = vagas['ies'].str.strip().str.lower() + \
                    vagas['campus'].str.strip().str.lower() + \
                    vagas['curso'].str.strip().str.lower() + \
                    "grau" + \
                    vagas['grau'].str.strip().str.lower() + \
                    "turno" + \
                    vagas['turno'].str.strip().str.lower()
cod_format = []
for i in vagas['cod_temp']:
    x = re.sub("[\s'\-\(\),/|]", '', i)
    cod_format.append(x)
vagas['cod_format'] = cod_format

In [9]:
# Normalização do curso para poder fundir com o df 'vagas' posteriormente
alunos['cod_temp'] = alunos['ies'].str.strip().str.lower() + \
                     alunos['campus'].str.strip().str.lower() + \
                     alunos['curso'].str.strip().str.lower() + \
                     alunos['grau_turno'].str.strip().str.lower()
cod_format = []
for i in alunos['cod_temp']:
    x = re.sub("[\s'\-\(\),/|]", '', i)
    cod_format.append(x)
alunos['cod_format'] = cod_format

In [10]:
# Chegagem da normalização do curso em ambos os dfs
alunos['cod_format'].isin(vagas['cod_format']).value_counts()

True    2765
Name: cod_format, dtype: int64

In [11]:
# Agrupamento por 'cod_format'
vagas_resumo = vagas.groupby(['cod_format']).agg({'numero_vagas': 'sum'}).reset_index()

In [12]:
# Checagem
vagas_resumo

,cod_format,numero_vagas
0,uspuniversidadedesãopaulocampusderibeirãopreto...,6
1,uspuniversidadedesãopauloescoladeartesciências...,18
2,uspuniversidadedesãopauloescoladeartesciências...,18
3,uspuniversidadedesãopauloescoladeartesciências...,18
4,uspuniversidadedesãopauloescoladeartesciências...,18
5,uspuniversidadedesãopauloescoladeartesciências...,18
6,uspuniversidadedesãopauloescoladeartesciências...,18
7,uspuniversidadedesãopauloescoladeartesciências...,18
8,uspuniversidadedesãopauloescoladeartesciências...,18
9,uspuniversidadedesãopauloescoladeartesciências...,18


In [13]:
# Agrupamento por 'cod_format'
alunos_resumo = alunos.groupby(['cod_format']).agg({'codigo_inscricao': 'count'}).reset_index()

In [14]:
# Checagem
alunos_resumo

,cod_format,codigo_inscricao
0,uspuniversidadedesãopaulocampusderibeirãopreto...,6
1,uspuniversidadedesãopauloescoladeartesciências...,18
2,uspuniversidadedesãopauloescoladeartesciências...,18
3,uspuniversidadedesãopauloescoladeartesciências...,18
4,uspuniversidadedesãopauloescoladeartesciências...,18
5,uspuniversidadedesãopauloescoladeartesciências...,18
6,uspuniversidadedesãopauloescoladeartesciências...,18
7,uspuniversidadedesãopauloescoladeartesciências...,18
8,uspuniversidadedesãopauloescoladeartesciências...,18
9,uspuniversidadedesãopauloescoladeartesciências...,18


In [15]:
# Fusão dos dois dfs a partir do 'cod_format'
final_resumo = vagas_resumo.merge(alunos_resumo, on='cod_format', how='left')

In [16]:
# Checagem
final_resumo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 0 to 171
Data columns (total 3 columns):
cod_format          172 non-null object
numero_vagas        172 non-null int64
codigo_inscricao    171 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.7+ KB


Um dos cursos não teve aluno aprovado (ou inscrito). Vamos ver qual.

In [17]:
# Checagem de valor nulo
final_resumo[final_resumo['codigo_inscricao'].isnull()]

,cod_format,numero_vagas,codigo_inscricao
5,uspuniversidadedesãopauloescoladeartesciências...,18,NaN


In [18]:
# Pesquisa no df 'vagas' se o curso existe e quantas vagas disponibilizou
vagas[vagas['cod_format'] == 'uspuniversidadedesãopauloescoladeartesciênciasehumanidadeseachsãopaulospgestãoambientalgraubachareladoturnomatutino']

,uf,ies,campus,curso,grau,turno,numero_vagas,tipo_modalidade,modalidade,cod_temp,cod_format
276,SP,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Bacharelado,Matutino,6,ação afirmativa,"autodeclarados pretos, pardos ou indígenas qu...",usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
277,SP,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Bacharelado,Matutino,6,ação afirmativa,"que independentemente da renda, tenham cursad...",usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
304,SP,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Bacharelado,Matutino,6,ampla concorrência,ampla concorrência.,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...


In [19]:
# Pesquisa no df 'alunos' quantos foram aprovados em gestão ambiental
alunos[alunos['curso'] == 'GESTÃO AMBIENTAL']

,estado,cidade,ies,campus,curso,grau_turno,modalidade,colocacao,nome,codigo_inscricao,nota,bonus,cod_temp,cod_format
431,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,Ampla concorrência.,1º,LAURA RITTNER SILVEIRA,18****466195,754.739990,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
432,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,Ampla concorrência.,2º,GABRIEL CAVAZZANA DE AGUIAR,18****085353,754.460022,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
433,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,Ampla concorrência.,3º,FELIPE DIEGO DA CUNHA GENTIL,18****127009,752.130005,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
434,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,Ampla concorrência.,4º,GABRIEL URBA,18****684200,752.090027,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
435,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,Ampla concorrência.,5º,LETICIA BAPTISTA DE MATTOS PIRES FERREIRA,18****857240,748.070007,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
436,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,Ampla concorrência.,6º,FERNANDA DE ALMEIDA E SILVA,18****573125,747.159973,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
437,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,"autodeclarados pretos, pardos ou indígenas que...",1º,ARIANY BEZERRA BRAZ,18****698705,692.969971,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
438,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,"autodeclarados pretos, pardos ou indígenas que...",2º,PEDRO HENRIQUE MARTINS BISPO,18****045750,684.400024,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
439,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,"autodeclarados pretos, pardos ou indígenas que...",3º,LUCAS BARROSO SAMPAIO,18****662392,684.070007,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...
440,SP,São Paulo,USP - UNIVERSIDADE DE SÃO PAULO,"ESCOLA DE ARTES, CIÊNCIAS E HUMANIDADES - EACH...",GESTÃO AMBIENTAL,Grau Bacharelado | Turno Noturno,"autodeclarados pretos, pardos ou indígenas que...",4º,MAYARA DOS SANTOS COSTA,18****599951,683.690002,NaN,usp - universidade de são pauloescola de artes...,uspuniversidadedesãopauloescoladeartesciências...


__Nenhum aluno__ foi aprovado em Gestão Ambiental no turno da manhã. Por isso há valor nulo.

In [20]:
# Criação da coluna 'nao_preenchida'
final_resumo['nao_preenchidas'] = final_resumo['numero_vagas'] - final_resumo['codigo_inscricao']

In [21]:
# Descarte de colunas inúteis
final_resumo.columns = ['cod', 'abertas', 'preenchidas', 'nao_preenchidas']

In [22]:
# Checagem de quais cursos não foram preenchidos
final_resumo[final_resumo['nao_preenchidas'] > 0]

,cod,abertas,preenchidas,nao_preenchidas
46,uspuniversidadedesãopauloescoladeengenhariades...,5,4.0,1.0
51,uspuniversidadedesãopauloescolapolitécnicaepsã...,5,4.0,1.0


In [23]:
# Registro de resultados
final_resumo.to_csv('usp_vagas_nao_preenchidas.csv', encoding='utf-8', index=False)